In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# filename = '/kaggle/input/mxmh-survey-results/mxmh_survey_results.csv'
#filename = '/content/drive/MyDrive/Applied ML Group 18/mxmh_survey_results.csv'
filename = '/content/drive/MyDrive/Applied ML Group 18/mxmh_survey_results.csv'
data = pd.read_csv(filename)
data

,Timestamp,Age,Primary streaming service,Hours per day,While working,Instrumentalist,Composer,Fav genre,Exploratory,Foreign languages,...,Frequency [R&B],Frequency [Rap],Frequency [Rock],Frequency [Video game music],Anxiety,Depression,Insomnia,OCD,Music effects,Permissions
0,8/27/2022 19:29:02,18.0,Spotify,3.0,Yes,Yes,Yes,Latin,Yes,Yes,...,Sometimes,Very frequently,Never,Sometimes,3.0,0.0,1.0,0.0,NaN,I understand.
1,8/27/2022 19:57:31,63.0,Pandora,1.5,Yes,No,No,Rock,Yes,No,...,Sometimes,Rarely,Very frequently,Rarely,7.0,2.0,2.0,1.0,NaN,I understand.
2,8/27/2022 21:28:18,18.0,Spotify,4.0,No,No,No,Video game music,No,Yes,...,Never,Rarely,Rarely,Very frequently,7.0,7.0,10.0,2.0,No effect,I understand.
3,8/27/2022 21:40:40,61.0,YouTube Music,2.5,Yes,No,Yes,Jazz,Yes,Yes,...,Sometimes,Never,Never,Never,9.0,7.0,3.0,3.0,Improve,I understand.
4,8/27/2022 21:54:47,18.0,Spotify,4.0,Yes,No,No,R&B,Yes,No,...,Very frequently,Very frequently,Never,Rarely,7.0,2.0,5.0,9.0,Improve,I understand.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,10/30/2022 14:37:28,17.0,Spotify,2.0,Yes,Yes,No,Rock,Yes,Yes,...,Never,Rarely,Very frequently,Never,7.0,6.0,0.0,9.0,Improve,I understand.
732,11/1/2022 22:26:42,18.0,Spotify,1.0,Yes,Yes,No,Pop,Yes,Yes,...,Never,Never,Sometimes,Sometimes,3.0,2.0,2.0,5.0,Improve,I understand.
733,11/3/2022 23:24:38,19.0,Other streaming service,6.0,Yes,No,Yes,Rap,Yes,No,...,Sometimes,Sometimes,Rarely,Rarely,2.0,2.0,2.0,2.0,Improve,I understand.
734,11/4/2022 17:31:47,19.0,Spotify,5.0,Yes,Yes,No,Classical,No,No,...,Never,Never,Never,Sometimes,2.0,3.0,2.0,1.0,Improve,I understand.


In [4]:
# look for the columns that have NaN/empty values
print(data.isna().sum())
# get the number of rows with NaN values
nan_values = data[data.isna().any(axis=1)]
print("Number of rows with missing values: ", len(nan_values)) # total is 134 rows

Timestamp                         0
Age                               1
Primary streaming service         1
Hours per day                     0
While working                     3
Instrumentalist                   4
Composer                          1
Fav genre                         0
Exploratory                       0
Foreign languages                 4
BPM                             107
Frequency [Classical]             0
Frequency [Country]               0
Frequency [EDM]                   0
Frequency [Folk]                  0
Frequency [Gospel]                0
Frequency [Hip hop]               0
Frequency [Jazz]                  0
Frequency [K pop]                 0
Frequency [Latin]                 0
Frequency [Lofi]                  0
Frequency [Metal]                 0
Frequency [Pop]                   0
Frequency [R&B]                   0
Frequency [Rap]                   0
Frequency [Rock]                  0
Frequency [Video game music]      0
Anxiety                     

In [5]:
#preprocessing
#Since the dataset is small, we will use mean, mode and KNN imputation to fill missing values instead of dropping the rows.
#We can drop timestamp and permissions columns as it doesn't really account for any predictions.

data['Age'].fillna(data['Age'].mean().astype(int), inplace=True)
categorical_columns = ['Primary streaming service', 'Composer', 'Instrumentalist', 'While working', 'Foreign languages']

for col in categorical_columns:
    mode_value = data[col].mode()[0]
    data[col].fillna(mode_value, inplace=True)


data.dropna(subset=["Music effects"], inplace=True)
data

,Timestamp,Age,Primary streaming service,Hours per day,While working,Instrumentalist,Composer,Fav genre,Exploratory,Foreign languages,...,Frequency [R&B],Frequency [Rap],Frequency [Rock],Frequency [Video game music],Anxiety,Depression,Insomnia,OCD,Music effects,Permissions
2,8/27/2022 21:28:18,18.0,Spotify,4.0,No,No,No,Video game music,No,Yes,...,Never,Rarely,Rarely,Very frequently,7.0,7.0,10.0,2.0,No effect,I understand.
3,8/27/2022 21:40:40,61.0,YouTube Music,2.5,Yes,No,Yes,Jazz,Yes,Yes,...,Sometimes,Never,Never,Never,9.0,7.0,3.0,3.0,Improve,I understand.
4,8/27/2022 21:54:47,18.0,Spotify,4.0,Yes,No,No,R&B,Yes,No,...,Very frequently,Very frequently,Never,Rarely,7.0,2.0,5.0,9.0,Improve,I understand.
5,8/27/2022 21:56:50,18.0,Spotify,5.0,Yes,Yes,Yes,Jazz,Yes,Yes,...,Very frequently,Very frequently,Very frequently,Never,8.0,8.0,7.0,7.0,Improve,I understand.
6,8/27/2022 22:00:29,18.0,YouTube Music,3.0,Yes,Yes,No,Video game music,Yes,Yes,...,Rarely,Never,Never,Sometimes,4.0,8.0,6.0,0.0,Improve,I understand.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,10/30/2022 14:37:28,17.0,Spotify,2.0,Yes,Yes,No,Rock,Yes,Yes,...,Never,Rarely,Very frequently,Never,7.0,6.0,0.0,9.0,Improve,I understand.
732,11/1/2022 22:26:42,18.0,Spotify,1.0,Yes,Yes,No,Pop,Yes,Yes,...,Never,Never,Sometimes,Sometimes,3.0,2.0,2.0,5.0,Improve,I understand.
733,11/3/2022 23:24:38,19.0,Other streaming service,6.0,Yes,No,Yes,Rap,Yes,No,...,Sometimes,Sometimes,Rarely,Rarely,2.0,2.0,2.0,2.0,Improve,I understand.
734,11/4/2022 17:31:47,19.0,Spotify,5.0,Yes,Yes,No,Classical,No,No,...,Never,Never,Never,Sometimes,2.0,3.0,2.0,1.0,Improve,I understand.


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 728 entries, 2 to 735
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Timestamp                     728 non-null    object 
 1   Age                           728 non-null    float64
 2   Primary streaming service     728 non-null    object 
 3   Hours per day                 728 non-null    float64
 4   While working                 728 non-null    object 
 5   Instrumentalist               728 non-null    object 
 6   Composer                      728 non-null    object 
 7   Fav genre                     728 non-null    object 
 8   Exploratory                   728 non-null    object 
 9   Foreign languages             728 non-null    object 
 10  BPM                           624 non-null    float64
 11  Frequency [Classical]         728 non-null    object 
 12  Frequency [Country]           728 non-null    object 
 13  Frequency 

In [7]:
# drop columns we don't need
columns_to_drop = ['Timestamp', 'Permissions', 'BPM']

if 'BPM' in data:
  data.drop(columns=columns_to_drop, inplace=True)

In [8]:
enc = OrdinalEncoder(categories = [['Never', 'Rarely', 'Sometimes', 'Very frequently']])
freq_cols = ['Frequency [Classical]',
       'Frequency [Country]', 'Frequency [EDM]', 'Frequency [Folk]',
       'Frequency [Gospel]', 'Frequency [Hip hop]', 'Frequency [Jazz]',
       'Frequency [K pop]', 'Frequency [Latin]', 'Frequency [Lofi]',
       'Frequency [Metal]', 'Frequency [Pop]', 'Frequency [R&B]',
       'Frequency [Rap]', 'Frequency [Rock]', 'Frequency [Video game music]']
for col in freq_cols:
    data[f'{col} ordinal'] = enc.fit_transform(data[col].to_numpy().reshape(-1,1))

bin_enc = OrdinalEncoder(categories = [['Yes', 'No', np.nan]])
bin_cols = ['While working', 'Instrumentalist', 'Composer', 'Exploratory']
for col in bin_cols:
    data[f'{col} ordinal'] = bin_enc.fit_transform(data[col].to_numpy().reshape(-1,1))

mapping = {'Worsen':-1, 'No effect':0, 'Improve':1}
data['Music effects ordinal'] = data['Music effects'].replace(mapping)

cat_cols = ['Primary streaming service', 'Fav genre', 'Foreign languages']
ohe_df = pd.get_dummies(data[cat_cols]).astype(int)
data1 = data.merge(ohe_df, left_index = True, right_index = True)

all_encoded_cols = freq_cols + bin_cols + ['Music effects'] + cat_cols
data1 = data1.drop(columns = all_encoded_cols)

data1.head()

,Age,Hours per day,Anxiety,Depression,Insomnia,OCD,Frequency [Classical] ordinal,Frequency [Country] ordinal,Frequency [EDM] ordinal,Frequency [Folk] ordinal,...,Fav genre_Latin,Fav genre_Lofi,Fav genre_Metal,Fav genre_Pop,Fav genre_R&B,Fav genre_Rap,Fav genre_Rock,Fav genre_Video game music,Foreign languages_No,Foreign languages_Yes
2,18.0,4.0,7.0,7.0,10.0,2.0,0.0,0.0,3.0,0.0,...,0,0,0,0,0,0,0,1,0,1
3,61.0,2.5,9.0,7.0,3.0,3.0,2.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,1
4,18.0,4.0,7.0,2.0,5.0,9.0,0.0,0.0,1.0,0.0,...,0,0,0,0,1,0,0,0,1,0
5,18.0,5.0,8.0,8.0,7.0,7.0,1.0,2.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
6,18.0,3.0,4.0,8.0,6.0,0.0,2.0,0.0,1.0,2.0,...,0,0,0,0,0,0,0,1,0,1


In [9]:
cat_cols = ['Primary streaming service', 'Foreign languages']
ohe_df = pd.get_dummies(data[cat_cols]).astype(int)
data2 = data.merge(ohe_df, left_index = True, right_index = True)

all_encoded_cols = freq_cols + bin_cols + ['Music effects'] + cat_cols
data2 = data2.drop(columns = all_encoded_cols)

enc2 = OrdinalEncoder(categories = [data2['Fav genre'].unique()])

data2['Fav genre'] = enc2.fit_transform(data2[['Fav genre']])

data2

,Age,Hours per day,Fav genre,Anxiety,Depression,Insomnia,OCD,Frequency [Classical] ordinal,Frequency [Country] ordinal,Frequency [EDM] ordinal,...,Exploratory ordinal,Music effects ordinal,Primary streaming service_Apple Music,Primary streaming service_I do not use a streaming service.,Primary streaming service_Other streaming service,Primary streaming service_Pandora,Primary streaming service_Spotify,Primary streaming service_YouTube Music,Foreign languages_No,Foreign languages_Yes
2,18.0,4.0,0.0,7.0,7.0,10.0,2.0,0.0,0.0,3.0,...,1.0,0,0,0,0,0,1,0,0,1
3,61.0,2.5,1.0,9.0,7.0,3.0,3.0,2.0,0.0,0.0,...,0.0,1,0,0,0,0,0,1,0,1
4,18.0,4.0,2.0,7.0,2.0,5.0,9.0,0.0,0.0,1.0,...,0.0,1,0,0,0,0,1,0,1,0
5,18.0,5.0,1.0,8.0,8.0,7.0,7.0,1.0,2.0,0.0,...,0.0,1,0,0,0,0,1,0,0,1
6,18.0,3.0,0.0,4.0,8.0,6.0,0.0,2.0,0.0,1.0,...,0.0,1,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,17.0,2.0,4.0,7.0,6.0,0.0,9.0,3.0,1.0,0.0,...,0.0,1,0,0,0,0,1,0,0,1
732,18.0,1.0,8.0,3.0,2.0,2.0,5.0,1.0,1.0,0.0,...,0.0,1,0,0,0,0,1,0,0,1
733,19.0,6.0,9.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,...,0.0,1,0,0,1,0,0,0,1,0
734,19.0,5.0,10.0,2.0,3.0,2.0,1.0,3.0,0.0,0.0,...,1.0,1,0,0,0,0,1,0,1,0


In [14]:
# split data into features and labels
cols_to_predict = ['Anxiety', 'Depression', 'Insomnia', 'OCD']
X_data1 = data1.drop(columns=cols_to_predict)
y_data1 = data1[cols_to_predict].to_numpy() # creates an array for each row of 4 columns

X_data2 = data2.drop(columns=['Fav genre'])
y_data2 = data2['Fav genre']

# X_data2 = data.drop(columns=['Fav genre'])
# y_data2 = data['Fav genre']


# split the dataset to get the test set and initial training set
X_train1, X_test1, y_train1, y_test1 = train_test_split(
    X_data1,
    y_data1,
    train_size=0.80,
    test_size=0.20,
    random_state=42
)


X_train2, X_test2, y_train2, y_test2 = train_test_split(
    X_data2,
    y_data2,
    train_size=0.80,
    test_size=0.20,
    stratify=y_data2,
    random_state=42
)

# print("Training set shape for Model 1: ", X_train1.shape)
# print("Test set shape for Model 1: ", X_test1.shape)


print("Training set shape for Model 2: ", X_train2.shape)
print("Test set shape for Model 2: ", X_test2.shape)

Training set shape for Model 2:  (582, 35)
Test set shape for Model 2:  (146, 35)


In [15]:
# SMOTE

# sm1 = SMOTE(random_state=42)
# X_train1, y_train1 = sm1.fit_resample(X_train1, y_train1)

sm2 = SMOTE(random_state=42, k_neighbors=1)
X_train2, y_train2 = sm2.fit_resample(X_train2, y_train2)

X_train2 = X_train2.astype(int)
X_train2

,Age,Hours per day,Anxiety,Depression,Insomnia,OCD,Frequency [Classical] ordinal,Frequency [Country] ordinal,Frequency [EDM] ordinal,Frequency [Folk] ordinal,...,Exploratory ordinal,Music effects ordinal,Primary streaming service_Apple Music,Primary streaming service_I do not use a streaming service.,Primary streaming service_Other streaming service,Primary streaming service_Pandora,Primary streaming service_Spotify,Primary streaming service_YouTube Music,Foreign languages_No,Foreign languages_Yes
0,22,2,7,7,8,3,1,0,1,1,...,1,1,0,0,0,0,0,1,1,0
1,18,2,5,7,2,0,2,1,2,0,...,0,1,0,0,0,0,1,0,1,0
2,31,2,8,9,6,2,1,1,0,1,...,0,1,0,0,0,0,1,0,1,0
3,17,3,8,10,9,2,1,1,0,2,...,0,1,1,0,0,0,0,0,1,0
4,18,3,8,9,0,0,1,0,0,2,...,0,1,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2379,17,8,4,4,4,2,3,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2380,17,10,1,3,7,4,3,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2381,17,9,3,4,5,3,3,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2382,17,10,2,3,6,3,3,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
y_train2

0        8.0
1        8.0
2        4.0
3        8.0
4        9.0
        ... 
2379    15.0
2380    15.0
2381    15.0
2382    15.0
2383    15.0
Name: Fav genre, Length: 2384, dtype: float64

In [17]:
LABEL_MAP = pd.DataFrame({'Fav genre': enc2.categories_[0]})
LABEL_MAP.index.names = ['Encoded Value']

LABEL_MAP

,Fav genre
Encoded Value,
0,Video game music
1,Jazz
2,R&B
3,K pop
4,Rock
5,Country
6,EDM
7,Hip hop
8,Pop


In [18]:
MODEL_DATA_DIR = "/content/drive/MyDrive/Applied ML Group 18/model_data"

X_TRAIN_2_FILE = "x_train_2.csv"
Y_TRAIN_2_FILE = "y_train_2.csv"
X_TEST_2_FILE = "x_test_2.csv"
Y_TEST_2_FILE = "y_test_2.csv"
LABEL_MAP_FILE = "label_map.csv"

X_train2.to_csv(X_TRAIN_2_FILE)
y_train2.to_csv(Y_TRAIN_2_FILE)
X_test2.to_csv(X_TEST_2_FILE)
y_test2.to_csv(Y_TEST_2_FILE)
LABEL_MAP.to_csv(LABEL_MAP_FILE)

!cp x_train_2.csv "/content/drive/MyDrive/Applied ML Group 18/model_data"
!cp y_train_2.csv "/content/drive/MyDrive/Applied ML Group 18/model_data"
!cp x_test_2.csv "/content/drive/MyDrive/Applied ML Group 18/model_data"
!cp y_test_2.csv "/content/drive/MyDrive/Applied ML Group 18/model_data"
!cp label_map.csv "/content/drive/MyDrive/Applied ML Group 18/model_data"

In [ ]:
# import matplotlib.pyplot as plt

# # Plotting the distribution of the target variable 'Fav genre'
# plt.figure(figsize=(8, 6))
# data2['Fav genre'].value_counts().plot(kind='bar')
# plt.xlabel('Genre')
# plt.ylabel('Count')
# plt.title('Distribution of Fav genre')
# plt.show()

In [ ]:
# # Plotting the correlation matrix
# corr_matrix = data2.corr()

# plt.figure(figsize=(10, 8))
# plt.imshow(corr_matrix, cmap='coolwarm', aspect='auto')
# plt.colorbar()
# plt.xticks(range(len(corr_matrix.columns)), corr_matrix.columns, rotation=90)
# plt.yticks(range(len(corr_matrix.columns)), corr_matrix.columns)
# plt.title('Correlation Matrix')
# plt.show()

In [ ]:
# # Plotting the class distribution before and after SMOTE
# plt.figure(figsize=(10, 4))

# plt.subplot(1, 2, 1)
# plt.bar(y_data2.value_counts().index, y_data2.value_counts().values)
# plt.xlabel('Genre')
# plt.ylabel('Count')
# plt.title('Class Distribution Before SMOTE')

# plt.subplot(1, 2, 2)
# plt.bar(np.unique(y_train2, return_counts=True)[0], np.unique(y_train2, return_counts=True)[1])
# plt.xlabel('Genre')
# plt.ylabel('Count')
# plt.title('Class Distribution After SMOTE')

# plt.tight_layout()
# plt.show()